# Q and A System

Install Necessary packages(torch, transformers and accelerate). Execution not needed if they have already been installed.

In [ ]:
pip install torch

In [ ]:
pip install transformers

In [ ]:
pip install accelerate

Code to import the downloaded model form local directory.

In [1]:
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration

# Define the path to your modified model directory
model_path = "./t5-small-text-to-sql/"

# Initialize the tokenizer from Hugging Face Transformers library
tokenizer = T5Tokenizer.from_pretrained(model_path)

# Load the model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = T5ForConditionalGeneration.from_pretrained(model_path)
model = model.to(device)
model.eval()


C:\Users\Anikool\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=512, bias=False)
              (k): Linear(in_features=512, out_features=512, bias=False)
              (v): Linear(in_features=512, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=512, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=512, bias=False)
              (dropout): Drop

Or alternatively, we can fetch the model from huggingface. Note that if the limitation on size is not present we can use a much better and more accurate model(defog/sqlcoder-7b-2 with 7 billion parameters or PipableAI/pip-sql-1.3b with 1.3 billion parameters).

In [ ]:
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration

# Initialize the tokenizer from Hugging Face Transformers library
tokenizer = T5Tokenizer.from_pretrained('t5-small')

# Load the model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = T5ForConditionalGeneration.from_pretrained('cssupport/t5-small-awesome-text-to-sql')
#T5 is fast but inaccurate
model = model.to(device)
model.eval()

Now, we can simply generate the required response. The details of the two tables have been provided with every prompt. Since the generated responses are SQL queries, we can convert them to the given format using some post processing. Running the cell below every time a new output is to be generated, we can convert standard english to the required format.

In [6]:
def generate_sql(input_prompt):
    # Tokenize the input prompt
    inputs = tokenizer(input_prompt, padding=True, truncation=True, return_tensors="pt")

    # Forward pass
    with torch.no_grad():
      outputs = model.generate(**inputs, max_length=512)

    # Decode the output IDs to a string (SQL query in this case)
    generated_sql = tokenizer.decode(outputs[0], skip_special_tokens=True)

    return generated_sql

x=input('Enter Query: ')
# Test the function
input_prompt = """
tables:
CREATE TABLE grid_details (
    ship_name VARCHAR(255), --Name of ship
    grid_name VARCHAR(255), --Name of grid
    grid_version VARCHAR(10), --Version
    grid_uptime INT, -- Uptime
    grid_time_since_version_change INT, --last version change
    grid_ram INT, -- RAM
    grid_last_version VARCHAR(10) --Last version
);

CREATE TABLE ship_data (
    ship_name VARCHAR(255), -- Name of ship
    ship_priority INT, -- Priority of ship
    is_hypercare INT, -- Flag for problem status
    active_pop VARCHAR(255), -- Active population on ship
    next_port VARCHAR(255), -- Next port destination
    ship_time VARCHAR(5), -- Current time on ship
    latest_message VARCHAR(255) -- Latest message received
);

query for:
""" + x
generated_query = generate_sql(input_prompt)

print(f"{generated_query}")


SELECT Priority FROM ship WHERE ship_name ='shiop nebula'
